In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

#### final cleaning of final.csv

In [15]:
df = pd.read_csv("db/sample/clean/final.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
print(df.info())

df2 = pd.DataFrame(columns=["word","definition"], index=None)
df2 = df[6576:]
df = df[:6575]

print(df2.info())
special_chars = []

def get_special_chars(defi):
    defi = str(defi)
    for d in list(defi):
        if (ord(d) < 48 and ord(d) != 32) or (ord(d) > 57 and ord(d)<65) or (ord(d) >90 and ord(d) < 97) or ord(d) > 122:
            if d not in special_chars:
                special_chars.append(d)
    return

df["definition"].apply(get_special_chars)

# special_chars.remove(";")

print(special_chars)

def remove_special_chars(defi):
    defi= str(defi)
    for s in special_chars:
        defi = " ".join(defi.split(s))
    index = 0
    defi = list(defi)
    while index < len(defi)-1:
        if defi[index] == " " and defi[index+1] == " ":
            del defi[index]
        else:
            index += 1
    return "".join(defi)


df["definition"] = df["definition"].apply(remove_special_chars)
df.to_csv("db/sample/clean/final_final.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="w")
df2.to_csv("db/sample/clean/final_final.csv", sep=":", columns=["word","definition"], header=False, index=False, mode="a")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9656 entries, 0 to 9655
Data columns (total 2 columns):
word          9656 non-null object
definition    6576 non-null object
dtypes: object(2)
memory usage: 151.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 6576 to 9655
Data columns (total 2 columns):
word          3080 non-null object
definition    0 non-null object
dtypes: object(2)
memory usage: 48.2+ KB
None
['(', ')', "'"]


#### calc vector position of 1 for all words in word column

In [16]:
df = pd.read_csv("db/sample/clean/final_final.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])

df_nums = pd.DataFrame(columns=["word","number"], index=None)
count = 0;

def word_vector_nums(row):
    global count, df_nums
    df_nums = df_nums.append({"word":row.word,"number": count}, ignore_index=True)
    count += 1
    return ""

df.apply(word_vector_nums, axis=1)
df_nums.to_csv("db/sample/clean/word_col_vectors.csv", sep=":", columns=["word","number"], header=False, index=False, mode="w")

#### convert definitions to vector positions

In [17]:
df = pd.read_csv("db/sample/clean/final_final.csv", names=["word","definition"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df_words = pd.read_csv("db/sample/clean/word_col_vectors.csv", names=["word","number"], sep=":", index_col=None, keep_default_na=False, na_values=[""])
df_defs = pd.DataFrame(columns=["word", "word_vector", "definition" ,"def_vectors"], index=None)

def def_vector_nums(row):
    global df_defs
    definition = str(row.definition)
    if (definition is not None and row.word is not None) or (definitions != "" and row.word != ""):
        word_vector = df_words.loc[df_words["word"] == row.word]["number"].values[0]
        vector =  " ".join(str(df_words.loc[df_words["word"] == word]["number"].values[0]) for word in definition.split())
        df_defs = df_defs.append({"word":row.word, "def_vectors":vector, "word_vector":word_vector, "definition": definition}, ignore_index=True)
    return

def fill_null(row):
    global df_defs
    df_defs = df_defs.append({"word":row.word, "word_vector":row.number, "definition":"-", "def_vectors": "-"}, ignore_index=True)
    return


df_2 = df[:6575]
df_3 = df_words[6576:]

print("*"*30)
print("df_2")
print(df_2.info())
print("*"*30)
print("df_3")
print(df_3.info())    

df_2.apply(def_vector_nums, axis=1)
df_3.apply(fill_null, axis=1)

print("*"*30)
print("df_defs")
print(df_defs.info())

df_defs.to_csv("db/sample/clean/def_col_vectors.csv", sep=":", columns=["word","word_vector", "definition" ,"def_vectors"], header=False, index=False, mode="w")

******************************
df_2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6575 entries, 0 to 6574
Data columns (total 2 columns):
word          6575 non-null object
definition    6575 non-null object
dtypes: object(2)
memory usage: 102.8+ KB
None
******************************
df_3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3078 entries, 6576 to 9653
Data columns (total 2 columns):
word      3078 non-null object
number    3078 non-null int64
dtypes: int64(1), object(1)
memory usage: 48.2+ KB
None
******************************
df_defs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9653 entries, 0 to 9652
Data columns (total 4 columns):
word           9653 non-null object
word_vector    9653 non-null object
definition     9653 non-null object
def_vectors    9653 non-null object
dtypes: object(4)
memory usage: 301.7+ KB
None
